In [25]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
from time import sleep

from tqdm.notebook import trange, tqdm
import pandas as pd

In [2]:
search_string = "amazfit t-rex pro"

In [3]:
search_string = search_string.replace(" ", "%20")
search_string

'amazfit%20t-rex%20pro'

In [8]:
# create object for chrome options
chrome_options = Options()
# base_url = 'https://shopee.com.my/shop/13377506/search?page=0&sortBy=sales'
base_url = 'https://shopee.com.my/search?keyword='+search_string

# set chrome driver options to disable any popup's from the website
# to find local path for chrome profile, open chrome browser
# and in the address bar type, "chrome://version"
chrome_options.add_argument('disable-notifications')
chrome_options.add_argument('--disable-infobars')
chrome_options.add_argument('start-maximized')
chrome_options.add_argument('user-data-dir=C:\\Users\\jimmy\\AppData\\Local\\Google\\Chrome\\User Data\\Default')
# To disable the message, "Chrome is being controlled by automated test software"
chrome_options.add_argument("disable-infobars")
# Pass the argument 1 to allow and 2 to block
chrome_options.add_experimental_option("prefs", { 
    "profile.default_content_setting_values.notifications": 2
    })


In [10]:
# invoke the webdriver
browser = webdriver.Chrome(executable_path = '../chromedriver.exe',
                          options = chrome_options)
browser.get(base_url)
delay = 5 #secods

In [11]:
def extract_element(item, class_name):
    value = ''
    try:
        value = item.find_element_by_class_name(class_name).text
    except:
        value = 'N/A'        
    
    return value
    

In [29]:
# invoke the webdriver
browser = webdriver.Chrome(executable_path = '../chromedriver.exe',
                          options = chrome_options)
browser.get(base_url)
delay = 5 #secods

sleep(10)

max_page = 5
current_page = 1

item_list = pd.DataFrame(columns=['name', 'promo_tag', 'initial_price', 
                                  'current_price', 'rating', 'sales_rate', 
                                  'ship_from', 'link'])
while current_page <= max_page:
    
    body =  browser.find_element_by_css_selector('.nt-s.nl-l')
    
    for j in range(0,5):
        body.send_keys(Keys.PAGE_DOWN)
        sleep(1)
        j +=1
    
    items =  browser.find_elements_by_class_name('shopee-search-item-result__item')
        
#     idx = 0
    
    pbar = tqdm(total=len(items))
    print("Page ", current_page)
    for item in items:
        
        pbar.update(1)
#         print(idx)
        temp = pd.DataFrame(columns=['name', 'promo_tag', 'initial_price', 
                                      'current_price', 'rating', 'sales_rate', 
                                      'ship_from', 'link'])
        stage = 0
        try:     
            item_name = extract_element(item, 'PFM7lj')        
            stage = 1

            item_promo_tag = extract_element(item, '_2NfTth').split('\n')    
            stage = 2

            item_initial_price = extract_element(item, '_3_-SiN')    
            stage = 3

            item_promo_price = extract_element(item, '_5W0f35')
            stage = 4

            star = 0
            rating = item.find_element_by_class_name('_3dC36C')
            for i in rating.find_elements_by_class_name('shopee-rating-stars__lit'):
                star = star +float(i.get_attribute('style').strip(' ').split(':')[1][:-2])
            item_rating = star/100        
            stage = 5

            item_sales = extract_element(item, 'go5yPW')        
            stage = 6

            item_ship_from = extract_element(item, '_2CWevj')
            stage = 7

            item_link = item.find_element_by_tag_name('a').get_attribute('href')

            temp = pd.DataFrame([[item_name, item_promo_tag, item_initial_price, item_promo_price, item_rating, 
    item_sales, item_ship_from, item_link]], columns=['name', 'promo_tag', 'initial_price', 
                                      'current_price', 'rating', 'sales_rate', 
                                      'ship_from', 'link'])
        except:
            print('problem')
            print(stage)
            break;

        item_list = item_list.append(temp)
#         idx += 1
    
    pbar.close()
#     browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(3)
    btn = browser.find_element_by_css_selector('.shopee-icon-button.shopee-icon-button--right')
    btn.click()
    current_page += 1
    sleep(delay)

browser.close()

  0%|          | 0/50 [00:00<?, ?it/s]

Page  1


  0%|          | 0/50 [00:00<?, ?it/s]

Page  2


  0%|          | 0/50 [00:00<?, ?it/s]

Page  3


  0%|          | 0/50 [00:00<?, ?it/s]

Page  4


  0%|          | 0/50 [00:00<?, ?it/s]

Page  5


In [30]:
item_list.tail()

,name,promo_tag,initial_price,current_price,rating,sales_rate,ship_from,link
0,∈✻(New)Amazfit T-Rex Pro Fitness Smartwatch-Gl...,[Local Seller],"RM1,993.00",RM996.50,0.0,,Kuala Lumpur,https://shopee.com.my/%E2%88%88%E2%9C%BB(New)A...
0,(USE VOUCHER RM70 DISCOUNT) Amazfit T Rex Pro ...,[N/A],N/A,RM969.50 - RM999.50,0.0,,Selangor,https://shopee.com.my/(USE-VOUCHER-RM70-DISCOU...
0,fast charging~travel charger~ [ High Quality ]...,[RM1 off],RM73.00,RM36.50,0.0,,Selangor,https://shopee.com.my/fast-charging~travel-cha...
0,Amazfit T-Rex / T-Rex Pro Huami Smart watch 47...,[Local Seller],N/A,"RM833.10 - RM1,297.06",0.0,,Kuala Lumpur,https://shopee.com.my/Amazfit-T-Rex-T-Rex-Pro-...
0,smart watch~smart watch women~ Amazfit T-Rex &...,"[RM1 off, Local Seller]",N/A,RM25.85 - RM30.25,0.0,,Selangor,https://shopee.com.my/smart-watch~smart-watch-...


In [31]:
len(item_list)

250

In [33]:
item_list

,name,promo_tag,initial_price,current_price,rating,sales_rate,ship_from,link
0,【PREMIUM】Amazfit T-Rex Pro 1.3 inch HD AMOLED ...,"[RM30 off, Local Seller]",RM659.00,RM629.00,4.975172,955 sold,Selangor,https://shopee.com.my/%E3%80%90PREMIUM%E3%80%9...
0,Amazfit T-Rex / T-Rex Pro Huami Smart watch 47...,"[Local Seller, COD]",N/A,RM409.00 - RM639.00,4.932016,1.8k sold,Selangor,https://shopee.com.my/Amazfit-T-Rex-T-Rex-Pro-...
0,Amazfit T-Rex Pro 1.3 inch HD AMOLED Screen / ...,"[Local Seller, COD]",RM659.00,RM639.00,4.891566,105 sold,Selangor,https://shopee.com.my/Amazfit-T-Rex-Pro-1.3-in...
0,[MY] Huami Amazfit T-Rex Pro Smartwatch - Orig...,"[RM10 off, Local Seller]",N/A,RM659.00,5.000000,3 sold,Selangor,https://shopee.com.my/-MY-Huami-Amazfit-T-Rex-...
0,#RAYAPROMO [OFFICIAL AMAZFIT MSIA] Amazfit T-R...,"[RM1 off, Local Seller]",RM659.00,RM638.00,5.000000,6 sold,Kuala Lumpur,https://shopee.com.my/-RAYAPROMO-OFFICIAL-AMAZ...
...,...,...,...,...,...,...,...,...
0,∈✻(New)Amazfit T-Rex Pro Fitness Smartwatch-Gl...,[Local Seller],"RM1,993.00",RM996.50,0.000000,,Kuala Lumpur,https://shopee.com.my/%E2%88%88%E2%9C%BB(New)A...
0,(USE VOUCHER RM70 DISCOUNT) Amazfit T Rex Pro ...,[N/A],N/A,RM969.50 - RM999.50,0.000000,,Selangor,https://shopee.com.my/(USE-VOUCHER-RM70-DISCOU...
0,fast charging~travel charger~ [ High Quality ]...,[RM1 off],RM73.00,RM36.50,0.000000,,Selangor,https://shopee.com.my/fast-charging~travel-cha...
0,Amazfit T-Rex / T-Rex Pro Huami Smart watch 47...,[Local Seller],N/A,"RM833.10 - RM1,297.06",0.000000,,Kuala Lumpur,https://shopee.com.my/Amazfit-T-Rex-T-Rex-Pro-...
